# Lab 04: Quantum Noise and Decoherence

In this lab, you will explore the effects of noise and decoherence on quantum circuits. Unlike classical computers, quantum computers are extremely sensitive to environmental interactions, which can lead to errors. Understanding these noise effects is crucial for developing error mitigation strategies and quantum error correction techniques.

## Learning Objectives
After completing this lab, you should be able to:
1. Understand different types of quantum noise models
2. Implement and simulate noisy quantum circuits
3. Visualize the effects of noise on the Bloch sphere
4. Measure the impact of noise on quantum algorithm performance
5. Apply basic error mitigation techniques

## Prerequisites
- Basic understanding of quantum circuits and gates
- Familiarity with the Bloch sphere representation of qubits
- Experience with quantum measurement and probabilities

## Exercise 1: Setup and Environment Preparation

First, let's import the necessary libraries and set up our quantum environment.

In [ ]:
# Import necessary libraries
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram, plot_bloch_multivector, plot_bloch_vector
from qiskit.quantum_info import Statevector
from qiskit_aer.noise import NoiseModel, pauli_error, depolarizing_error
from qiskit.quantum_info import state_fidelity

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Enable inline plotting
%matplotlib inline

## Exercise 2: Simulating a Perfect vs. Noisy Quantum Circuit

Let's start by comparing the behavior of a simple quantum circuit with and without noise. This will help us understand the basic impact of noise on measurement results.

**Task**: Create a simple quantum circuit that prepares a qubit in the |0⟩ state and then measures it. Run the circuit on both a perfect simulator and a noisy simulator, and compare the results.

In [ ]:
# YOUR CODE HERE: Create a quantum circuit with one qubit and one classical bit

# YOUR CODE HERE: Add a measurement operation

# YOUR CODE HERE: Draw the circuit

# Run on a perfect simulator
# YOUR CODE HERE: Run the circuit on a perfect simulator with 1000 shots

# Create a custom noise model
# YOUR CODE HERE: Create a noise model with bit-flip error on measurements
# The error probability should be 5% (0.05)

# Run with the noise model
# YOUR CODE HERE: Run the circuit with the noise model

# Display and compare the results
# YOUR CODE HERE: Plot and analyze results from both simulations

### Understanding Bit-Flip Noise

In the noise model you created above, you used a Pauli-X error to model bit-flip noise. Let's understand what this means:

1. **Bit-Flip Noise**: This type of noise randomly flips the state of a qubit from |0⟩ to |1⟩ or vice versa.
   - Mathematically represented by applying a Pauli-X gate with some probability
   - The notation `pauli_error([('X', 0.05), ('I', 0.95)])` means:
     - With 5% probability: Apply Pauli-X gate (bit flip)
     - With 95% probability: Apply Identity operation (no change)

2. **Real-World Causes of Bit-Flip Errors**:
   - Environmental noise affecting qubit state
   - Imperfect control pulses during gate operations
   - Readout errors during measurement

## Exercise 3: Noise Effects on Superposition

Now, let's see how noise affects quantum circuits that involve superposition.

**Task**: Create a circuit that applies a Hadamard gate to put a qubit into superposition, then measures it. Compare the results with and without noise.

In [ ]:
# YOUR CODE HERE: Create a circuit with Hadamard gate and measurement

# YOUR CODE HERE: Draw the circuit

# Create a noise model with both bit-flip and depolarizing noise
# YOUR CODE HERE: Create a noise model with:
# - 5% bit-flip error on measurements
# - 2% depolarizing error on single-qubit gates

# Run on perfect and noisy simulators
# YOUR CODE HERE: Run on both simulators and compare results

# Display and compare the results

# Analyze how far the noisy results deviate from the ideal 50-50 split
# YOUR CODE HERE: Calculate and print probability analysis

### Understanding Depolarizing Noise

In your enhanced noise model, you added depolarizing error to single-qubit gates. Let's understand what this means:

1. **Depolarizing Noise**: This type of noise randomly transforms a qubit's state toward the maximally mixed state (the center of the Bloch sphere).
   - With probability p: The qubit is replaced with a completely mixed state
   - With probability (1-p): The qubit remains unchanged

2. **Mathematical Representation**: A depolarizing channel with probability p can be described as:
   - With probability (1-p): The qubit state remains unchanged
   - With probability p/3: The X gate is applied
   - With probability p/3: The Y gate is applied
   - With probability p/3: The Z gate is applied

3. **Visual Effect**: On the Bloch sphere, depolarizing noise causes the state vector to shrink toward the center.

## Exercise 4: Visualizing Depolarizing Noise on the Bloch Sphere

Let's visualize how depolarizing noise affects quantum states on the Bloch sphere.

**Task**: Create a function to apply depolarizing noise to a quantum state and visualize the effect on the Bloch sphere.

In [ ]:
# YOUR CODE HERE: Define a function to apply depolarizing noise
def apply_depolarizing_noise(initial_state, noise_strength):
    """
    Apply depolarizing noise to a quantum state
    
    Args:
        initial_state: Initial quantum state vector [x, y, z]
        noise_strength: Value between 0 and 1 indicating noise strength
        
    Returns:
        Noisy state vector [x', y', z']
    """
    # YOUR CODE HERE: Implement the function
    pass

# YOUR CODE HERE: Choose an initial state to visualize
# For example, the |+⟩ state: [1, 0, 0]

# YOUR CODE HERE: Apply noise at different strengths and visualize

## Exercise 5: Phase-Flip Noise

In addition to bit-flip and depolarizing noise, another important type of quantum noise is phase-flip noise, which randomly changes the phase of a qubit.

**Task**: Create a noise model with phase-flip errors and observe its effect on a superposition state.

In [ ]:
# YOUR CODE HERE: Create a noise model with phase-flip errors

# Create a circuit to test phase-flip noise
# The circuit should create a superposition, then apply the noise,
# then apply another Hadamard to convert phase difference to bit difference
# YOUR CODE HERE: Create and run the circuit

# Run with phase-flip noise

# Compare with a circuit without noise
# YOUR CODE HERE: Run the same circuit without noise and compare

# Display the results

### Understanding Phase-Flip Noise

1. **Phase-Flip Noise**: This type of noise randomly changes the phase of a qubit.
   - Mathematically represented by applying a Pauli-Z gate with some probability
   - The notation `pauli_error([('Z', 0.05), ('I', 0.95)])` means:
     - With 5% probability: Apply Pauli-Z gate (phase flip)
     - With 95% probability: Apply Identity operation (no change)

2. **Effect on Superposition**: Phase-flip noise is particularly damaging to quantum algorithms because it disrupts the phase relationships that are crucial for quantum interference.

3. **Detection**: Phase-flip errors can't be detected by direct measurement in the computational basis. We need to convert phase information to bit information (e.g., by applying a Hadamard gate) before measurement.

## Exercise 6: Noise Effects on a Bell State

Now let's see how noise affects entangled states, which are even more sensitive to noise than single-qubit states.

**Task**: Create a circuit that prepares a Bell state, and observe how different types of noise affect the entanglement.

In [ ]:
# YOUR CODE HERE: Create a circuit that prepares a Bell state

# YOUR CODE HERE: Draw the circuit

# Create different noise models
# YOUR CODE HERE: Create noise models with bit-flip, phase-flip, and depolarizing noise

# 1. Bit-flip noise model

# 2. Phase-flip noise model

# 3. Depolarizing noise model

# Run the Bell state circuit with different noise models
# YOUR CODE HERE: Run simulations and compare results

# Display the results

### Understanding Noise Effects on Entanglement

Entangled states like Bell states are particularly sensitive to noise for several reasons:

1. **Non-local Effects**: Noise affecting just one qubit of an entangled pair can affect the joint state of both qubits.

2. **Decoherence**: Noise causes quantum states to lose their coherence, which is essential for maintaining entanglement.

3. **Measurement Statistics**: In a perfect Bell state, the measurement outcomes of the two qubits should be perfectly correlated. Noise disrupts this correlation.

## Exercise 7: Error Mitigation with Repetition

One simple approach to mitigating quantum errors is to run the circuit multiple times and take the average result. This can help reduce the impact of random noise.

**Task**: Implement a simple error mitigation technique by running a noisy circuit multiple times and comparing the average result with the ideal result.

In [ ]:
# YOUR CODE HERE: Create a circuit for testing error mitigation

# YOUR CODE HERE: Create a noise model

# Run the circuit once with noise
# YOUR CODE HERE: Run the circuit once with 1000 shots

# Run the circuit multiple times with noise and average the results
# YOUR CODE HERE: Run the circuit 10 times with 100 shots each, then average

# Compare the results
# YOUR CODE HERE: Compare single-run results, averaged results, and ideal results

## Reflection Questions

1. How do bit-flip and phase-flip errors differ in terms of their effect on quantum states?

2. Why is quantum error correction more challenging than classical error correction?

3. How does depolarizing noise affect the Bloch sphere representation of a qubit?

4. Why might entangled states be more sensitive to noise than non-entangled states?

5. Can you think of any applications where quantum noise might actually be useful?

## Summary

In this lab, you have explored:
- Different types of quantum noise models (bit-flip, phase-flip, depolarizing)
- The effects of noise on single-qubit and multi-qubit circuits
- Visualization of noise effects on the Bloch sphere
- Basic error mitigation techniques

Understanding quantum noise and decoherence is crucial for the development of practical quantum computers. As quantum systems scale up, error correction and mitigation strategies become increasingly important for running reliable quantum algorithms.